In [37]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import theano

import matplotlib.pyplot as plt


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = pandas.read_csv("iris.csv", header=1)
dataset = dataframe.values

In [3]:
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [4]:
dataset[0]

array([4.9, 3.0, 1.4, 0.2, 'Iris-setosa'], dtype=object)

In [5]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# 4 inputs -> [4 hidden nodes] -> 3 outputs

In [11]:
# Create model in Keras
model = Sequential()
model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
model.add(Dense(3, init='normal', activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(X, dummy_y)

Epoch 1/10
149/149 [==============================] - 0s - loss: 1.0991 - acc: 0.3020     
Epoch 2/10
149/149 [==============================] - 0s - loss: 1.0989 - acc: 0.2752     
Epoch 3/10
149/149 [==============================] - 0s - loss: 1.0988 - acc: 0.2819     
Epoch 4/10
149/149 [==============================] - 0s - loss: 1.0987 - acc: 0.2013     
Epoch 5/10
149/149 [==============================] - 0s - loss: 1.0987 - acc: 0.2819     
Epoch 6/10
149/149 [==============================] - 0s - loss: 1.0986 - acc: 0.3087     
Epoch 7/10
149/149 [==============================] - 0s - loss: 1.0985 - acc: 0.3020     
Epoch 8/10
149/149 [==============================] - 0s - loss: 1.0985 - acc: 0.3020     
Epoch 9/10
149/149 [==============================] - 0s - loss: 1.0984 - acc: 0.2953     
Epoch 10/10
149/149 [==============================] - 0s - loss: 1.0983 - acc: 0.2953     


In [13]:
model.evaluate(X, dummy_y)

 32/149 [=====>........................] - ETA: 0s

[1.0981628038739197, 0.29530201402286554]

In [16]:
X[0]

array([ 4.9,  3. ,  1.4,  0.2])

In [34]:
model.predict_on_batch(X[0:1])

array([[ 0.49862155,  0.49925467,  0.4989669 ]], dtype=float32)

In [43]:
from keras.utils.visualize_util import plot
plot(model, to_file='iris_model.png')

In [44]:
all_weights = []

for layer in model.layers:
    weights = layer.get_weights()
    all_weights.append(weights)

In [45]:
all_weights[0]

[array([[ -8.87888670e-02,  -1.95538700e-02,  -1.19722057e-02,
          -5.01334146e-02],
        [  3.62569690e-02,   7.24487826e-02,   6.31183211e-05,
           3.50696109e-02],
        [ -4.07587476e-02,  -4.43844646e-02,   4.56509134e-03,
           3.56371850e-02],
        [ -8.09256919e-03,   7.79600888e-02,  -3.47055718e-02,
          -4.89966497e-02]], dtype=float32),
 array([ 0.        ,  0.00813074, -0.00596425, -0.00802436], dtype=float32)]

In [6]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
	model.add(Dense(3, init='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


In [7]:
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=200, batch_size=5, verbose=0)


In [9]:
# Evaluate the model with k-Fold Cross Validation
kfold = KFold(n=len(X), n_folds=10, shuffle=True, random_state=seed)


In [10]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 98.00% (3.06%)


In [35]:
all_weights = []
for layer in model:
   w = layer.get_weights()
   all_weights.append(w)
    
print(all_weights)

TypeError: 'Sequential' object is not iterable